In [11]:
# 1. Upload the Excel Dataset
from google.colab import files
import pandas as pd

uploaded = files.upload()

# Find the first uploaded .xlsx file
excel_file = next((fname for fname in uploaded if fname.endswith(".xlsx")), None)

if excel_file is None:
    raise FileNotFoundError("❌ No .xlsx file found in uploaded files.")

# 2. Load and Inspect the Dataset
df = pd.read_excel(excel_file)

# Clean column names for consistency
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

print(f"Loaded file: {excel_file}")
print("Column names:", df.columns.tolist())
display(df.head())

# Function for smart p-value formatting
def format_p(p):
    return f"{p:.4f}" if p >= 0.001 else f"{p:.2e}"

# 3. Analyze Pre- vs Post-Flight Vigilance (PVT) Scores
from scipy.stats import ttest_rel, wilcoxon, shapiro

shapiro_pre = shapiro(df["pvt_pre"].dropna())
shapiro_post = shapiro(df["pvt_post"].dropna())

if shapiro_pre.pvalue > 0.05 and shapiro_post.pvalue > 0.05:
    stat, p = ttest_rel(df["pvt_pre"], df["pvt_post"], nan_policy='omit')
    print("Paired t-test was used for PVT comparison.")
else:
    stat, p = wilcoxon(df["pvt_pre"], df["pvt_post"])
    print("Wilcoxon test was used for PVT comparison.")
print(f"PVT_pre vs PVT_post: p = {format_p(p)}")

# 4. Compare Subjective Fatigue (KSS and SP) Before vs After Flight
kss_p = ttest_rel(df["kss_pre"], df["kss_post"], nan_policy='omit').pvalue
sp_p = ttest_rel(df["sp_pre"], df["sp_post"], nan_policy='omit').pvalue
print(f"KSS_pre vs KSS_post: p = {format_p(kss_p)}")
print(f"SP_pre  vs SP_post:  p = {format_p(sp_p)}")

# 5. Correlation and Regression Analysis (with PVT_avr)
from scipy.stats import pearsonr
import statsmodels.api as sm

# Correlation with PVT_post
r_kss, _ = pearsonr(df["pvt_post"], df["kss_post"])
r_sp, _  = pearsonr(df["pvt_post"], df["sp_post"])
r_avr, _ = pearsonr(df["pvt_post"], df["pvt_avr"])

print(f"Correlation (PVT_post vs KSS_post): r = {r_kss:.3f}")
print(f"Correlation (PVT_post vs SP_post):  r = {r_sp:.3f}")
print(f"Correlation (PVT_post vs PVT_avr):  r = {r_avr:.3f}")

# Regression model
X = df[["pvt_pre", "pvt_avr", "kss_post", "sp_post", "flight_hours"]].copy()
X = sm.add_constant(X)
y = df["pvt_post"]
model = sm.OLS(y, X, missing='drop').fit()
print("Linear Regression Results:\n")
print(model.summary())

# 6. Compare Fatigue Scores by Flight Type (within-subject comparison)
print("\n Fatigue Comparison by Mission Type (within-subject):")

try:
    df["flight_group"] = df["flight_type"].replace({
        "first_officer": "operational",
        "op_pilot": "operational"
    })

    matched_pilots = df.groupby("pilot_id")["flight_group"].nunique()
    matched_pilots = matched_pilots[matched_pilots == 2].index.tolist()
    df_matched = df[df["pilot_id"].isin(matched_pilots)]

    pivot = df_matched.pivot(index="pilot_id", columns="flight_group", values=["pvt_post", "kss_post", "sp_post"])
    pivot = pivot.dropna()

    def compare_metric(metric_name):
        a = pivot[(metric_name, "ab_initio")]
        b = pivot[(metric_name, "operational")]
        if len(a) < 3:
            print(f"⚠️ Not enough data for {metric_name}")
            return
        try:
            p1 = shapiro(a).pvalue
            p2 = shapiro(b).pvalue
            if p1 > 0.05 and p2 > 0.05:
                stat, pval = ttest_rel(a, b)
                method = "Paired t-test"
            else:
                stat, pval = wilcoxon(a, b)
                method = "Wilcoxon"
            print(f"{metric_name}: {method} p = {format_p(pval)}")
        except Exception as e:
            print(f"Error in {metric_name}: {e}")

    compare_metric("kss_post")
    compare_metric("sp_post")
    compare_metric("pvt_post")

except Exception as e:
    print(f"Error during within-subject mission type comparison: {e}")


Saving dataset.xlsx to dataset (2).xlsx
Loaded file: dataset (2).xlsx
Column names: ['pilot_id', 'flight_type', 'pvt_pre', 'pvt_post', 'pvt_avr', 'kss_pre', 'kss_post', 'sp_pre', 'sp_post', 'flight_time_min', 'age', 'flight_year', 'flight_hours']


,pilot_id,flight_type,pvt_pre,pvt_post,pvt_avr,kss_pre,kss_post,sp_pre,sp_post,flight_time_min,age,flight_year,flight_hours
0,Inst_01,ab_initio,449.0,709.5,579.25,3,3,4,7,120,38,13,3500
1,Inst_01,first_officer,424.5,472.0,448.25,3,3,4,4,135,38,13,3500
2,Inst_02,ab_initio,402.0,630.5,516.25,2,2,2,6,135,43,20,4000
3,Inst_02,first_officer,406.0,452.0,429.00,4,4,3,3,140,43,20,4000
4,Inst_03,ab_initio,387.5,426.5,407.00,2,2,2,6,140,49,26,6500


Wilcoxon test was used for PVT comparison.
PVT_pre vs PVT_post: p = 1.82e-05
KSS_pre vs KSS_post: p = 0.0306
SP_pre  vs SP_post:  p = 2.33e-04
Correlation (PVT_post vs KSS_post): r = -0.047
Correlation (PVT_post vs SP_post):  r = 0.412
Correlation (PVT_post vs PVT_avr):  r = 0.962
Linear Regression Results:

                            OLS Regression Results                            
Dep. Variable:               pvt_post   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 4.696e+28
Date:                Sun, 01 Jun 2025   Prob (F-statistic):          2.26e-252
Time:                        17:19:53   Log-Likelihood:                 625.70
No. Observations:                  24   AIC:                            -1239.
Df Residuals:                      18   BIC:                            -1232.
Df Model:                           5                     